In [ ]:
!pip install googletrans==3.1.0a0

In [61]:
from googletrans import Translator
import pandas as pd

In [70]:
description=pd.read_csv('raw_data/Clustering_ProductDescription.csv', engine='python', error_bad_lines=False, encoding='utf8')

description = description.fillna("null")

<ipython-input-70-966d77bff555>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  description=pd.read_csv('raw_data/Clustering_ProductDescription.csv', engine='python', error_bad_lines=False, encoding='utf8')
Skipping line 4560: unexpected end of data


In [71]:
description

,ASIN,MARKETPLACE_NAME,SELLER_NAME,ITEM_DESCRIPTION
0,B0847TJHB1,United Kingdom,Espring,<b>&#10031&#10031 WINTEX Fabric Template Ruler...
1,B06XVJL3NP,United Kingdom,H&S,null
2,B01M8GI87J,Germany,Espring,<p><b>&#10031&#10031 PRETEX 5ER-SET MIKROFASER...
3,B08X1SQWL4,Italy,Tritart,null
4,B07D26LYLH,Poland,H&S,null
...,...,...,...,...
4553,B08ZC82Q8Q,United Kingdom,Espring,null
4554,B09JNWYNHZ,Sweden,Espring,<p><strong><strong>EVEREST FITNESS Luftpumpar ...
4555,B01I9P24BK,Sweden,Espring,null
4556,B017QDER5C,United Kingdom,H&S,Our high quality cardboard jewellery gift boxe...


In [72]:
translator = Translator()

lang_detect = lambda x : translator.detect(x).lang
to_english = lambda x: translator.translate(x, dest='en').text


In [73]:
#takes a while to execute. Haven't managed to optimize this line unfortunately.

description['detected_lang'] = description['ITEM_DESCRIPTION'].apply(lang_detect)

In [74]:
non_english_descriptions = description[description['detected_lang'] != "en"]
english_descriptions = description[description['detected_lang'] == "en"]
english_descriptions['detected_lang'] = "en"

<ipython-input-74-fafdf782c740>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_descriptions['detected_lang'] = "en"


In [75]:
#takes a while to execute too.

non_english_descriptions['ITEM_DESCRIPTION_ENG'] = non_english_descriptions['ITEM_DESCRIPTION'].apply(to_english)
non_english_descriptions2 = non_english_descriptions.drop(columns=['ITEM_DESCRIPTION']).rename(columns={"ITEM_DESCRIPTION_ENG":"ITEM_DESCRIPTION"})


<ipython-input-75-72bec7b4734e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_english_descriptions['ITEM_DESCRIPTION_ENG'] = non_english_descriptions['ITEM_DESCRIPTION'].apply(to_english)


In [77]:
descriptions2 = pd.concat([non_english_descriptions2,english_descriptions]).sort_index()

In [78]:
descriptions3 = descriptions2[descriptions2['ITEM_DESCRIPTION'] != "null"]

In [79]:
descriptions3

,ASIN,MARKETPLACE_NAME,SELLER_NAME,detected_lang,ITEM_DESCRIPTION
0,B0847TJHB1,United Kingdom,Espring,en,<b>&#10031&#10031 WINTEX Fabric Template Ruler...
2,B01M8GI87J,Germany,Espring,de,<p><b>&#10031&#10031 PRETEX SET OF 5 MICROFIBE...
12,B078X5N2K1,Netherlands,H&S,nl,H&S Foot Massager for Plantar Fasciitis and Ar...
14,B07PQQRNBM,Germany,Espring,de,<b>&#10031&#10031 GOURMEO MUFFIN CUPS &#10031&...
17,B00TY1HA18,United Kingdom,H&S,en,These swivel &amp; braked castors with 4 bolt ...
...,...,...,...,...,...
4546,B00BVEADWO,United Kingdom,H&S,en,A quality shoe tree must be made of natural so...
4551,B07X1YPMR2,United States,Espring,en,Ted Williams Men's Bow Tie Light Grey
4554,B09JNWYNHZ,Sweden,Espring,sv,<p><strong><strong>EVEREST FITNESS Air Pumps F...
4556,B017QDER5C,United Kingdom,H&S,en,Our high quality cardboard jewellery gift boxe...


In [80]:
#remove HTML Tags
descriptions3['ITEM_DESCRIPTION2'] = descriptions3['ITEM_DESCRIPTION'].str.replace(r'<[^<>]*>', '', regex=True)

<ipython-input-80-f09deefa878a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descriptions3['ITEM_DESCRIPTION2'] = descriptions3['ITEM_DESCRIPTION'].str.replace(r'<[^<>]*>', '', regex=True)


In [81]:
#keep word characters only
descriptions3['ITEM_DESCRIPTION3'] = descriptions3['ITEM_DESCRIPTION2'].str.replace(r'\W', ' ', regex=True)

<ipython-input-81-460fd3f08106>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descriptions3['ITEM_DESCRIPTION3'] = descriptions3['ITEM_DESCRIPTION2'].str.replace(r'\W', ' ', regex=True)


In [82]:
descriptions3['MARKETPLACE_NAME'].unique()

array(['United Kingdom', 'Germany', 'Netherlands', 'Italy',
       'United States', 'Spain', 'Sweden', 'Poland', 'Turkey', 'Canada',
       'Belgium'], dtype=object)

In [83]:
map_marketplace = dict(zip(['United Kingdom', 'Germany', 'Netherlands', 'Italy',
       'United States', 'Spain', 'Sweden', 'Poland', 'Turkey', 'Canada',
       'Belgium'],
    ['UK', 'DE', 'NL','IT','US', 'ES', 'SE','PL','TR','CA','BE']))

descriptions3['MARKETPLACE_NAME'] = descriptions3['MARKETPLACE_NAME'].map(map_marketplace)

<ipython-input-83-eae9e33158c3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descriptions3['MARKETPLACE_NAME'] = descriptions3['MARKETPLACE_NAME'].map(map_marketplace)


In [84]:
descriptions3 = descriptions3.rename(columns={'ASIN1':'ASIN',
                              'MARKETPLACE_NAME':"MARKETPLACE",
                              "SELLER_NAME":"BRAND_NAME",
                                            "ITEM_DESCRIPTION3":"DESCRIPTION"})


In [85]:
final_descriptions = descriptions3[['ASIN','MARKETPLACE','BRAND_NAME','DESCRIPTION']]

In [86]:
final_descriptions

,ASIN,MARKETPLACE,BRAND_NAME,DESCRIPTION
0,B0847TJHB1,UK,Espring,10031 10031 WINTEX Fabric Template Ruler ...
2,B01M8GI87J,DE,Espring,10031 10031 PRETEX SET OF 5 MICROFIBER CLOT...
12,B078X5N2K1,NL,H&S,H S Foot Massager for Plantar Fasciitis and Ar...
14,B07PQQRNBM,DE,Espring,10031 10031 GOURMEO MUFFIN CUPS 10031 10...
17,B00TY1HA18,UK,H&S,These swivel amp braked castors with 4 bolt ...
...,...,...,...,...
4546,B00BVEADWO,UK,H&S,A quality shoe tree must be made of natural so...
4551,B07X1YPMR2,US,Espring,Ted Williams Men s Bow Tie Light Grey
4554,B09JNWYNHZ,SE,Espring,EVEREST FITNESS Air Pumps For Bicycle Footbal...
4556,B017QDER5C,UK,H&S,Our high quality cardboard jewellery gift boxe...


In [ ]:
final_descriptions.to_csv('preprocessed_data/Product_Descriptions.csv',index=False)